In [1]:
import iris
import iris.coord_categorisation
from iris.experimental.equalise_cubes import equalise_attributes
import copy

In [2]:
path = '/g/data1/r87/drstree/CMIP5/GCM/CSIRO-QCCCE/CSIRO-Mk3-6-0/piControl/mon/ocean/thetao/r1i1p1/'
control_files_all = path+'thetao_Omon_CSIRO-Mk3-6-0_piControl_r1i1p1_*.nc'

In [3]:
cube = iris.load(control_files_all, 'sea_water_potential_temperature')
equalise_attributes(cube)
cube = cube.concatenate_cube()

/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'areacello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:794: UserWarning: Missing CF-netCDF measure variable u'volcello', referenced by netCDF variable u'thetao'
  warnings.warn(message % (variable_name, nc_var_name))
/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/fileformats/cf.py:1139: UserWarning: NetCDF default loading behaviour currently does not expose variables which define reference surfaces for dimensionless vertical coordinates as independent Cubes. This behaviour is deprecated in favour of automatic promotion to Cubes. To switch to the new behaviour, set iris.FUTURE.netcdf_promote to True.
  warnings.warn(msg)


In [4]:
print cube

sea_water_potential_temperature / (K) (time: 6000; depth: 31; latitude: 189; longitude: 192)
     Dimension coordinates:
          time                             x            -             -               -
          depth                            -            x             -               -
          latitude                         -            -             x               -
          longitude                        -            -             -               x
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Stephen.Jeffrey@qld.gov.au) & Leon Rotstayn...
          experiment: pre-industrial control
          experiment_id: piCo

### Annual climatology

In [4]:
annual_climatology = cube.collapsed('time', iris.analysis.MEAN)

In [5]:
print annual_climatology

sea_water_potential_temperature / (K) (depth: 31; latitude: 189; longitude: 192)
     Dimension coordinates:
          depth                             x             -               -
          latitude                          -             x               -
          longitude                         -             -               x
     Scalar coordinates:
          time:  251-01-01 00:00:00, bound=(   1-01-01 00:00:00,  501-01-01 00:00:00)
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Stephen.Jeffrey@qld.gov.au) & Leon Rotstayn...
          experiment: pre-industrial control
          experiment_id: piControl
          forcing: 

### Monthly climatology

In [7]:
iris.coord_categorisation.add_month(cube, 'time', name='month')

In [8]:
print cube

sea_water_potential_temperature / (K) (time: 6000; depth: 31; latitude: 189; longitude: 192)
     Dimension coordinates:
          time                             x            -             -               -
          depth                            -            x             -               -
          latitude                         -            -             x               -
          longitude                        -            -             -               x
     Auxiliary coordinates:
          month                            x            -             -               -
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Steph

In [9]:
monthly_climatology = cube.aggregated_by(['month'], iris.analysis.MEAN)

/g/data/r87/dbi599/miniconda2/envs/default/lib/python2.7/site-packages/iris/coords.py:495: VisibleDeprecationWarning: an index can only have a single Ellipsis (`...`); replace all but one with slices (`:`).
  bounds = bounds[keys + (Ellipsis, )]


MemoryError: Failed to create the cube's data as there was not enough memory available.
The array shape would have been (6000, 31, 189, 192) and the data type float32.
Consider freeing up variables or indexing the cube before getting its data.

In [11]:
coord_names = [coord.name() for coord in cube.coords(dim_coords=True)]
print coord_names

[u'time', u'depth', u'latitude', u'longitude']


In [13]:
if 'depth' in coord_names:
    slice_dims = copy.copy(coord_names)
    slice_dims.remove('depth')
    cube_list = []
    for i, x_slice in enumerate(cube.slices(slice_dims)):
        monthly_climatology = x_slice.aggregated_by(['month'], iris.analysis.MEAN)
        cube_list.append(monthly_climatology)

In [16]:
print cube_list[1]

sea_water_potential_temperature / (K) (time: 12; latitude: 189; longitude: 192)
     Dimension coordinates:
          time                             x             -               -
          latitude                         -             x               -
          longitude                        -             -               x
     Auxiliary coordinates:
          month                            x             -               -
     Scalar coordinates:
          depth: 15.0 m, bound=(10.0, 21.6242351532) m
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Stephen.Jeffrey@qld.gov.au) & Leon Rotstayn...
          experiment: pre-indus

In [17]:
new_cube = iris.cube.CubeList(cube_list)

In [19]:
new_cube = new_cube.merge_cube()

In [20]:
print new_cube

sea_water_potential_temperature / (K) (depth: 31; time: 12; latitude: 189; longitude: 192)
     Dimension coordinates:
          depth                             x         -             -               -
          time                              -         x             -               -
          latitude                          -         -             x               -
          longitude                         -         -             -               x
     Auxiliary coordinates:
          month                             -         x             -               -
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Stephen.Jeffrey@q

How do I change the order of the dimesions so that depth is second??

In [21]:
new_cube.transpose([1,0,2,3])

In [22]:
print new_cube

sea_water_potential_temperature / (K) (time: 12; depth: 31; latitude: 189; longitude: 192)
     Dimension coordinates:
          time                             x          -             -               -
          depth                            -          x             -               -
          latitude                         -          -             x               -
          longitude                        -          -             -               x
     Auxiliary coordinates:
          month                            x          -             -               -
     Attributes:
          Conventions: CF-1.4
          associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_ocean_fx_CSIRO-Mk3-6-0_piControl_r0i0p0.nc...
          branch_time: 0.0
          cmor_version: 2.5.9
          comment: Data is stored on the native ocean T-grid on which the data was generated....
          contact: Project leaders: Stephen Jeffrey (Stephen.Jeffrey@q

In [24]:
dims = range(0,4)
dims[0] = 1
dims[1] = 0
print dims

[1, 0, 2, 3]
